# DataFrames: Read and Write Data
     
Dask Dataframes can read and store data in many of the same formats as Pandas dataframes.  In this example we read and write data with the popular CSV and Parquet formats, and discuss best practices when using these formats.

In [1]:
from IPython.display import HTML

HTML('<iframe width="560" height="315" src="https://www.youtube.com/embed/0eEsIA0O1iE?rel=0&amp;controls=0&amp;showinfo=0" frameborder="0" allowfullscreen></iframe>')

/usr/share/miniconda3/envs/dask-examples/lib/python3.8/site-packages/IPython/core/display.py:717: UserWarning: Consider using IPython.display.IFrame instead
  warnings.warn("Consider using IPython.display.IFrame instead")


## Start Dask Client for Dashboard

Starting the Dask Client is optional.  It will provide a dashboard which 
is useful to gain insight on the computation.  

The link to the dashboard will become visible when you create the client below.  We recommend having it open on one side of your screen while using your notebook on the other side.  This can take some effort to arrange your windows, but seeing them both at the same is very useful when learning.

In [2]:
from dask.distributed import Client
client = Client(n_workers=1, threads_per_worker=4, processes=False, memory_limit='2GB')
client

Client Scheduler: inproc://10.1.0.4/9782/1 Dashboard: http://10.1.0.4:8787/status,Cluster Workers: 1 Cores: 4 Memory: 2.00 GB


## Create artificial dataset

First we create an artificial dataset and write it to many CSV files.

You don't need to understand this section, we're just creating a dataset for the rest of the notebook.

In [3]:
import dask
df = dask.datasets.timeseries()
df

,id,name,x,y
npartitions=30,,,,
2000-01-01,int64,object,float64,float64
2000-01-02,...,...,...,...
...,...,...,...,...
2000-01-30,...,...,...,...
2000-01-31,...,...,...,...


In [4]:
import os
import datetime

if not os.path.exists('data'):
    os.mkdir('data')

def name(i):
    """ Provide date for filename given index
    
    Examples
    --------
    >>> name(0)
    '2000-01-01'
    >>> name(10)
    '2000-01-11'
    """
    return str(datetime.date(2000, 1, 1) + i * datetime.timedelta(days=1))
    
df.to_csv('data/*.csv', name_function=name);

['/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-01.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-02.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-03.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-04.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-05.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-06.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-07.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-08.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-09.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-10.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-11.csv',
 '/home/runner/work/dask-examples/dask-examples/dataframes/data/2000-01-12.csv',
 '/home/runner/work/dask-exa

## Read CSV files

We now have many CSV files in our data directory, one for each day in the month of January 2000.  Each CSV file holds timeseries data for that day.  We can read all of them as one logical dataframe using the `dd.read_csv` function with a glob string.

In [5]:
!ls data/*.csv | head

data/2000-01-01.csv
data/2000-01-02.csv
data/2000-01-03.csv
data/2000-01-04.csv
data/2000-01-05.csv
data/2000-01-06.csv
data/2000-01-07.csv
data/2000-01-08.csv
data/2000-01-09.csv
data/2000-01-10.csv


In [6]:
!head data/2000-01-01.csv

timestamp,id,name,x,y
2000-01-01 00:00:00,997,Jerry,-0.598693888964215,-0.33867533108695924
2000-01-01 00:00:01,1025,Frank,0.8374970416821816,0.2730154114213468
2000-01-01 00:00:02,1065,Charlie,0.6274160566202722,0.09458162185917263
2000-01-01 00:00:03,985,Edith,-0.7815264147691583,-0.13623922932242327
2000-01-01 00:00:04,980,Xavier,0.8169816174361666,-0.8665600691749575
2000-01-01 00:00:05,953,Tim,0.6037074810925713,0.5248435997998275
2000-01-01 00:00:06,984,Quinn,-0.7202916067190699,-0.31958754717519966
2000-01-01 00:00:07,971,Charlie,0.9254134234875941,0.747743720447702
2000-01-01 00:00:08,985,Hannah,-0.4288854843527503,-0.8142682712007057


In [7]:
!head data/2000-01-30.csv

timestamp,id,name,x,y
2000-01-30 00:00:00,1004,Zelda,-0.7100029029192205,0.007080793218195991
2000-01-30 00:00:01,999,Victor,-0.3968957303684162,-0.8900645307981307
2000-01-30 00:00:02,999,Dan,0.7069081103388599,0.517098000844034
2000-01-30 00:00:03,988,Xavier,-0.6606775620100989,0.5506228469660439
2000-01-30 00:00:04,1054,Yvonne,0.7345088384454104,0.11957991051495798
2000-01-30 00:00:05,996,Xavier,0.7110394994360458,-0.3013309424337711
2000-01-30 00:00:06,996,Bob,0.4192152490520309,-0.134126514062199
2000-01-30 00:00:07,959,Alice,-0.2631793670333129,-0.23519264464066714
2000-01-30 00:00:08,990,Sarah,-0.41924755187218654,0.9716165979942593


We can read one file with `pandas.read_csv` or many files with `dask.dataframe.read_csv`

In [8]:
import pandas as pd

df = pd.read_csv('data/2000-01-01.csv')
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,997,Jerry,-0.598694,-0.338675
1,2000-01-01 00:00:01,1025,Frank,0.837497,0.273015
2,2000-01-01 00:00:02,1065,Charlie,0.627416,0.094582
3,2000-01-01 00:00:03,985,Edith,-0.781526,-0.136239
4,2000-01-01 00:00:04,980,Xavier,0.816982,-0.866560


In [9]:
import dask.dataframe as dd

df = dd.read_csv('data/2000-*-*.csv')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,object,int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [10]:
df.head()

,timestamp,id,name,x,y
0,2000-01-01 00:00:00,997,Jerry,-0.598694,-0.338675
1,2000-01-01 00:00:01,1025,Frank,0.837497,0.273015
2,2000-01-01 00:00:02,1065,Charlie,0.627416,0.094582
3,2000-01-01 00:00:03,985,Edith,-0.781526,-0.136239
4,2000-01-01 00:00:04,980,Xavier,0.816982,-0.866560


## Tuning read_csv

The Pandas `read_csv` function has *many* options to help you parse files.  The Dask version uses the Pandas function internally, and so supports many of the same options.  You can use the `?` operator to see the full documentation string.

In [11]:
pd.read_csv?

In [12]:
dd.read_csv?

In this case we use the `parse_dates` keyword to parse the timestamp column to be a datetime.  This will make things more efficient in the future.  Notice that the dtype of the timestamp column has changed from `object` to `datetime64[ns]`.

In [13]:
df = dd.read_csv('data/2000-*-*.csv', parse_dates=['timestamp'])
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


## Do a simple computation

Whenever we operate on our dataframe we read through all of our CSV data so that we don't fill up RAM.  This is very efficient for memory use, but reading through all of the CSV files every time can be slow.

In [14]:
%time df.groupby('name').x.mean().compute()

CPU times: user 4.05 s, sys: 432 ms, total: 4.48 s
Wall time: 3.12 s


name
Alice       0.000408
Bob        -0.000648
Charlie    -0.001565
Dan         0.001498
Edith      -0.000778
Frank      -0.001643
George     -0.001610
Hannah     -0.002178
Ingrid      0.000240
Jerry       0.001379
Kevin      -0.000849
Laura      -0.001792
Michael    -0.001407
Norbert     0.001216
Oliver      0.001654
Patricia   -0.002757
Quinn       0.002643
Ray        -0.000953
Sarah      -0.002452
Tim        -0.000243
Ursula      0.001349
Victor     -0.000345
Wendy      -0.003445
Xavier      0.001023
Yvonne     -0.000226
Zelda       0.002122
Name: x, dtype: float64

## Write to Parquet

Instead, we'll store our data in Parquet, a format that is more efficient for computers to read and write.

In [15]:
df.to_parquet('data/2000-01.parquet', engine='pyarrow')

In [16]:
!ls data/2000-01.parquet/

_common_metadata  part.14.parquet  part.21.parquet  part.29.parquet
_metadata	  part.15.parquet  part.22.parquet  part.3.parquet
part.0.parquet	  part.16.parquet  part.23.parquet  part.4.parquet
part.1.parquet	  part.17.parquet  part.24.parquet  part.5.parquet
part.10.parquet   part.18.parquet  part.25.parquet  part.6.parquet
part.11.parquet   part.19.parquet  part.26.parquet  part.7.parquet
part.12.parquet   part.2.parquet   part.27.parquet  part.8.parquet
part.13.parquet   part.20.parquet  part.28.parquet  part.9.parquet


## Read from Parquet

In [17]:
df = dd.read_parquet('data/2000-01.parquet', engine='pyarrow')
df

,timestamp,id,name,x,y
npartitions=30,,,,,
,datetime64[ns],int64,object,float64,float64
,...,...,...,...,...
...,...,...,...,...,...
,...,...,...,...,...
,...,...,...,...,...


In [18]:
%time df.groupby('name').x.mean().compute()

CPU times: user 1.26 s, sys: 168 ms, total: 1.43 s
Wall time: 1.06 s


name
Alice       0.000408
Bob        -0.000648
Charlie    -0.001565
Dan         0.001498
Edith      -0.000778
Frank      -0.001643
George     -0.001610
Hannah     -0.002178
Ingrid      0.000240
Jerry       0.001379
Kevin      -0.000849
Laura      -0.001792
Michael    -0.001407
Norbert     0.001216
Oliver      0.001654
Patricia   -0.002757
Quinn       0.002643
Ray        -0.000953
Sarah      -0.002452
Tim        -0.000243
Ursula      0.001349
Victor     -0.000345
Wendy      -0.003445
Xavier      0.001023
Yvonne     -0.000226
Zelda       0.002122
Name: x, dtype: float64

## Select only the columns that you plan to use

Parquet is a column-store, which means that it can efficiently pull out only a few columns from your dataset.  This is good because it helps to avoid unnecessary data loading.

In [19]:
%%time
df = dd.read_parquet('data/2000-01.parquet', columns=['name', 'x'], engine='pyarrow')
df.groupby('name').x.mean().compute()

CPU times: user 1.26 s, sys: 107 ms, total: 1.36 s
Wall time: 1.07 s


name
Alice       0.000408
Bob        -0.000648
Charlie    -0.001565
Dan         0.001498
Edith      -0.000778
Frank      -0.001643
George     -0.001610
Hannah     -0.002178
Ingrid      0.000240
Jerry       0.001379
Kevin      -0.000849
Laura      -0.001792
Michael    -0.001407
Norbert     0.001216
Oliver      0.001654
Patricia   -0.002757
Quinn       0.002643
Ray        -0.000953
Sarah      -0.002452
Tim        -0.000243
Ursula      0.001349
Victor     -0.000345
Wendy      -0.003445
Xavier      0.001023
Yvonne     -0.000226
Zelda       0.002122
Name: x, dtype: float64

Here the difference is not that large, but with larger datasets this can save a great deal of time.

## Learn more

http://dask.pydata.org/en/latest/dataframe-create.html